先將 data 做預處理，因為光只有 `open-high-low-close` 預測不出甚麼趨勢，所以我就將這四項資料任抓取某兩項的間格，最後再將原本的 `open-high-low-close` 刪掉，然後回傳 feature and target

In [6]:
def process(Data):
    value = pd.Series(Data['open'].shift(-2) - Data['open'].shift(-1), index = Data.index)
    Data['High-Low'] = Data['high'] - Data['low'] #today's High - Low
    Data['High-Close'] = Data['high'] - Data['close'] #today's High - Close
    Data['High-Open'] = Data['high'] - Data['open'] #today's High - Open
    Data['Close-Open'] = Data['close'] - Data['open'] #today's Close - Open
    Data['Close-High'] = Data['close'] - Data['high'] #today's Close - High
    Data['Close-Low'] = Data['close'] - Data['low'] #today's Close - Low
    Data['Open-Close'] = Data['open'] - Data['close'] #todat's Open - Close
    Data['Open-High'] = Data['open'] - Data['high'] #today's Open - High
    value = value.dropna()
    value[value >= 0] = 1 #0 means rise
    value[value < 0] = 0 #1 means fall
    del(Data['open'])
    del(Data['close'])
    del(Data['high'])
    del(Data['low'])
    return Data[:len(Data)-2], value

再利用 `sklearn` 中的 `cross_validation` 將原本的 training_data 分成一半一半下去做 training

In [7]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_x, train_y, test_size = 0.5, random_state = 0)
clf = svm.SVC(kernel='rbf', C=1).fit(X_train, y_train)

NameError: name 'train_x' is not defined

最後在 output action 時，避免掉錯誤情況的發生

In [ ]:
def get_action(array):
    global handle
    for index, x in np.ndenumerate(array):
        # print(index, x)
        if handle == 0:
            if x == 1:
                handle = 1
                return 1
            elif x == 0:
                handle = -1
                return -1
        elif handle == 1:
            if x == 1:
                return 0
            elif x == 0:
                handle = 0
                return -1
        elif handle == -1:
            if x == 1:
                handle = 0
                return 1
            elif x == 0:
                return 0